In [5]:
!pip install PyPDF2 enchant spacy

Defaulting to user installation because normal site-packages is not writeable
  Using cached spacy-3.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached murmurhash-1.0.9-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
  Using cached srsly-2.4.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (491 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached cymem-2.0.7-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35 kB)
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached thinc-8.1.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (922 kB)
  Using cached cat

In [8]:
!python3 -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd
import numpy as np
import enchant
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.cluster import KMeans 
import PyPDF2
import sys
import time
import spacy
import warnings
warnings.filterwarnings('ignore')
sys.path.append("..")
from modules import helper_functions as hf
from modules import similarity_functions as sf
# Load the en_core_web_sm model
nlp = spacy.load('en_core_web_sm')
stopwords = spacy.lang.en.stop_words.STOP_WORDS

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [10]:
# Load data
df_jobs = pd.read_csv("../data/df_job_final.csv", usecols=['title', 'department', 'description_combined'])
df_resume = pd.read_csv("../data/data_resume_cc.csv", usecols=['Category', 'Resume_c'])
# department mapping between job and resume labels
THRESHOLD = 50
df_jobs = hf.get_map_category(df_jobs, 'department', THRESHOLD )
df_resume = hf.get_map_category(df_resume, 'Category', THRESHOLD)
df_jobs_nan = df_jobs[df_jobs['department'].isna() == True]
df_jobs = df_jobs[df_jobs['department'].isna() == False]
# load models
vec = hf.load_tfidf('./pretrained/tfidf_job.pkl')
svm_clf = hf.load_tfidf('./pretrained/tfidf_clf.pkl')
kmeans = hf.load_tfidf('./pretrained/tfidf_cluster.pkl')
df_jobs['cluster'] = kmeans.labels_

In [17]:
# start processing resume
start = time.time()
pdffileobj = open('../data/resume_puck.pdf','rb')
pdfreader = PyPDF2.PdfReader(pdffileobj)
pages = len(pdfreader.pages)
resume = ''
for i in range(0, pages):
    pageobj = pdfreader.pages[i]
    resume = resume + pageobj.extract_text() + ' '

In [18]:
##lemmatize Resume
lemmas = [token.lemma_ for token in nlp(resume)]
resume = [lemma.lower() for lemma in lemmas if lemma.isalpha() and lemma not in stopwords or lemma == '-PRON-']
resume = ' '.join(resume)

In [19]:
pred_department = svm_clf.predict([resume])[0]
resume_matrix = vec.transform([resume])[:, :15240]
pred_cluster = kmeans.predict(resume_matrix)[0]

In [20]:
jobs_top_n = hf.get_top_n_jobs_from_clf(df_jobs=df_jobs,
                            pred_department=pred_department,
                            resume=resume,
                            vec=vec,
                            sim_func=sf.cal_cosine_similarity)
rec_clf = df_jobs.iloc[jobs_top_n]
rec_clf_filtered = rec_clf[rec_clf['department'] == pred_department]
rec_clf_filtered

,title,department,description_combined,cluster
1974,system administrator,it,"['-pron-', 'seek', 'system', 'administrator', ...",0


In [21]:
jobs_top_n = hf.get_top_n_jobs_from_cluster(df_jobs, pred_cluster, resume, vec, sf.cal_cosine_similarity)
rec_cluster = df_jobs.iloc[jobs_top_n]
rec_cluster_filtered = rec_cluster[rec_cluster['department'] == pred_department]
rec_cluster_filtered

,title,department,description_combined,cluster
219,senior software engineer - mobile,it,"['senior', 'software', 'engineer', 'highly', '...",0
2752,hardware expert,it,"['hardware', 'expert', 'm', 'mission', 'enable...",0
2834,sap-abap development,it,"['job', 'title', 'sap', 'abap', 'development',...",0
1185,interface developer,it,"['-pron-', 'seek', 'interface', 'end', 'develo...",0
2480,c# [.net framework],it,"['job', 'title', 'c', 'mandaluyong', 'experien...",0


In [22]:
# assume last 50% of the resumes got offers
applicant_pool_with_offer = df_resume[:(int)(len(df_resume)*0.5)]
application_pool = applicant_pool_with_offer[applicant_pool_with_offer['Category'] == pred_department]
# build resume similarity matrix
temp = application_pool['Resume_c'].append(pd.Series(resume))
matrix =vec.transform(temp)
term_matrix = matrix.todense()
cossim = sf.cal_cosine_similarity(term_matrix)
# get similar resumes based on given resume
index_similar_applicant = np.asarray(cossim[-1][np.where(cossim[-1] < 1)]).argsort()[::-1][:10]
cf_jobs = hf.get_top_n_jobs_from_cf(df_jobs, df_resume, index_similar_applicant, svm_clf, vec, sf.cal_cosine_similarity, 5)
# recommend jobs based on similar resumes
rec_from_cf = df_jobs.iloc[cf_jobs]
rec_from_cf_filtered = rec_from_cf[rec_from_cf['department'] == pred_department]

In [23]:
list_filtered_rec = [rec_clf_filtered, rec_cluster_filtered, rec_from_cf_filtered]
final_rec = pd.concat(list_filtered_rec)
final_rec = final_rec.drop_duplicates()
end = time.time()
final_rec

,title,department,description_combined,cluster
1974,system administrator,it,"['-pron-', 'seek', 'system', 'administrator', ...",0
219,senior software engineer - mobile,it,"['senior', 'software', 'engineer', 'highly', '...",0
2752,hardware expert,it,"['hardware', 'expert', 'm', 'mission', 'enable...",0
2834,sap-abap development,it,"['job', 'title', 'sap', 'abap', 'development',...",0
1185,interface developer,it,"['-pron-', 'seek', 'interface', 'end', 'develo...",0
2480,c# [.net framework],it,"['job', 'title', 'c', 'mandaluyong', 'experien...",0
1248,frontend ux / magento themes developer,it,"['referral', 'bonus', '-pron-', 'offer', 'succ...",0
448,web developer (full stack),it,"['incrediblue', 'change', 'people', 'experienc...",0
1997,quality & support manager,it,"['-pron-', 'seek', 'bilingual', 'french', 'eng...",0
1991,internship (information technology),it,"['time', 'pay', 'internship', 'position', 'lar...",0


In [24]:
print(f'Time spent: {end-start}')

Time spent: 14.363134622573853
